In [ ]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = SparkSession.builder \
    .appName("ECommerceAnalytics") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
orders_df = spark.read.option("header", "true").csv("/content/drive/MyDrive/June17_18/orders.csv")
customers_df = spark.read.option("header", "true").csv("/content/drive/MyDrive/June17_18/customers.csv")
products_df = spark.read.option("header", "true").csv("/content/drive/MyDrive/June17_18/products.csv")

+-------+----------+---------+--------+-----+----------+---------+
|OrderID|CustomerID|ProductID|Quantity|Price| OrderDate|   Status|
+-------+----------+---------+--------+-----+----------+---------+
|   3001|      C001|    P1001|       1|75000|2024-05-01|Delivered|
|   3002|      C002|    P1002|       2|50000|2024-05-02| Returned|
|   3003|      C003|    P1003|       1|30000|2024-05-03|Delivered|
|   3004|      C001|    P1002|       1|50000|2024-05-04|Delivered|
|   3005|      C004|    P1004|       3|10000|2024-05-05|  Pending|
+-------+----------+---------+--------+-----+----------+---------+



# **PySpark + Delta**

**1. Ingest all 3 CSVs as Delta Tables.**

In [9]:
orders_df.write.format("delta").mode("overwrite").saveAsTable("orders_delta")
customers_df.write.format("delta").mode("overwrite").saveAsTable("customers_delta")
products_df.write.format("delta").mode("overwrite").saveAsTable("products_delta")


**2. Write SQL to get the total revenue per Product.**

In [13]:
spark.sql("""select p.ProductID, p.ProductName, sum(o.Quantity * o.Price) as TotalRevenue
from orders_delta o join products_delta p ON o.ProductID = p.ProductID
where o.Status != 'Returned' group by p.ProductID, p.ProductName
order by TotalRevenue DESC""").show()

+---------+-----------+------------+
|ProductID|ProductName|TotalRevenue|
+---------+-----------+------------+
|    P1001|     Laptop|     75000.0|
|    P1002|      Phone|     50000.0|
|    P1004|   Keyboard|     30000.0|
|    P1003|     Tablet|     30000.0|
+---------+-----------+------------+



**3. Join Orders + Customers to find revenue by Region.**

In [17]:
spark.sql("""select c.Region, sum(o.Quantity * o.Price) as RegionalRevenue
from orders_delta o join customers_delta c ON o.CustomerID = c.CustomerID
where o.Status != 'Returned' group by c.Region
order by RegionalRevenue DESC""").show()

+------+---------------+
|Region|RegionalRevenue|
+------+---------------+
| North|       125000.0|
|  East|        30000.0|
|  West|        30000.0|
+------+---------------+



**4. Update Status of Pending orders to 'Cancelled'**

In [22]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forName(spark, "orders_delta")
delta_table.update(condition = "Status = 'Pending'",set = {"Status": "'Cancelled'"})
spark.sql("SELECT * FROM orders_delta").show()

+-------+----------+---------+--------+-----+----------+---------+
|OrderID|CustomerID|ProductID|Quantity|Price| OrderDate|   Status|
+-------+----------+---------+--------+-----+----------+---------+
|   3001|      C001|    P1001|       1|75000|2024-05-01|Delivered|
|   3002|      C002|    P1002|       2|50000|2024-05-02| Returned|
|   3003|      C003|    P1003|       1|30000|2024-05-03|Delivered|
|   3004|      C001|    P1002|       1|50000|2024-05-04|Delivered|
|   3005|      C004|    P1004|       3|10000|2024-05-05|Cancelled|
+-------+----------+---------+--------+-----+----------+---------+



**5. Merge a new return record into Orders**

In [25]:
new_return = spark.createDataFrame([(3006, "C003", "P1001", 1, 52500, "2025-01-06", "Returned")], ["OrderID", "CustomerID", "ProductID", "Quantity", "Price", "OrderDate", "Status"])

DeltaTable.forName(spark, "orders_delta").alias("orders").merge(new_return.alias("updates"),"orders.OrderID = updates.OrderID") \
  .whenMatchedUpdateAll() \
  .whenNotMatchedInsertAll() \
  .execute()

spark.sql("SELECT * FROM orders_delta").show()

+-------+----------+---------+--------+-----+----------+---------+
|OrderID|CustomerID|ProductID|Quantity|Price| OrderDate|   Status|
+-------+----------+---------+--------+-----+----------+---------+
|   3001|      C001|    P1001|       1|75000|2024-05-01|Delivered|
|   3002|      C002|    P1002|       2|50000|2024-05-02| Returned|
|   3003|      C003|    P1003|       1|30000|2024-05-03|Delivered|
|   3004|      C001|    P1002|       1|50000|2024-05-04|Delivered|
|   3005|      C004|    P1004|       3|10000|2024-05-05|Cancelled|
|   3006|      C003|    P1001|       1|52500|2025-01-06| Returned|
+-------+----------+---------+--------+-----+----------+---------+



# **DLT Pipeline**

**6. DLT Pipeline (raw → cleaned → aggregated)**

In [32]:
from pyspark.sql.functions import col, sum

raw_orders = spark.read.option("header", "true").csv("orders.csv")
cleaned_orders = raw_orders.filter((col("Quantity") > 0) &(col("Price") > 0) &col("OrderDate").isNotNull() &col("Status").isNotNull())
products = spark.read.format("delta").table("products_delta")
aggregated_revenue = cleaned_orders.join(products, "ProductID") \
    .groupBy("Category") \
    .agg(sum(col("Quantity") * col("Price")).alias("TotalRevenue"))

aggregated_revenue.show()

+-----------+------------+
|   Category|TotalRevenue|
+-----------+------------+
|Electronics|    255000.0|
|Accessories|     30000.0|
+-----------+------------+



# **Time Travel**

**7. View data before the Status update.**

In [55]:
from delta.tables import DeltaTable
current_version = DeltaTable.forName(spark, "orders_delta").history() \
                          .selectExpr("max(version)").first()[0]
spark.read.format("delta") \
    .option("versionAsOf", current_version - 3) \
    .table("orders_delta") \
    .show()

+-------+----------+---------+--------+-----+----------+---------+
|OrderID|CustomerID|ProductID|Quantity|Price| OrderDate|   Status|
+-------+----------+---------+--------+-----+----------+---------+
|   3001|      C001|    P1001|       1|75000|2024-05-01|Delivered|
|   3002|      C002|    P1002|       2|50000|2024-05-02| Returned|
|   3003|      C003|    P1003|       1|30000|2024-05-03|Delivered|
|   3004|      C001|    P1002|       1|50000|2024-05-04|Delivered|
|   3005|      C004|    P1004|       3|10000|2024-05-05|  Pending|
+-------+----------+---------+--------+-----+----------+---------+



**8. Restore to older version**

In [59]:
delta_table.restoreToVersion(current_version-3)
spark.sql("select * from orders_delta").show()

+-------+----------+---------+--------+-----+----------+---------+
|OrderID|CustomerID|ProductID|Quantity|Price| OrderDate|   Status|
+-------+----------+---------+--------+-----+----------+---------+
|   3001|      C001|    P1001|       1|75000|2024-05-01|Delivered|
|   3002|      C002|    P1002|       2|50000|2024-05-02| Returned|
|   3003|      C003|    P1003|       1|30000|2024-05-03|Delivered|
|   3004|      C001|    P1002|       1|50000|2024-05-04|Delivered|
|   3005|      C004|    P1004|       3|10000|2024-05-05|  Pending|
+-------+----------+---------+--------+-----+----------+---------+



# **Vacuum + Retention**

**9. Run VACUUM after changing default retention.**

In [60]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")
spark.sql("""alter table orders_delta
SET TBLPROPERTIES ('delta.logRetentionDuration' = '7 days','delta.deletedFileRetentionDuration' = '7 days')""")

spark.sql("VACUUM orders_delta RETAIN 0 HOURS")

DataFrame[path: string]

# **Expectations**

**10. Quantity > 0 , Price > 0 , OrderDate is not null**

In [62]:
spark.sql("""select count(*) as total_orders,sum(case when quantity > 0 and price > 0 and orderdate is not null then 1 else 0 end) as valid_orders,sum(case when quantity <= 0 or price <= 0 or orderdate is null then 1 else 0 end) as invalid_orders
from orders_delta""").show()

+------------+------------+--------------+
|total_orders|valid_orders|invalid_orders|
+------------+------------+--------------+
|           5|           5|             0|
+------------+------------+--------------+



# **Bonus**

**11. Use when-otherwise to create a new column: OrderType = "Return" if Status ==
'Returned'**

In [63]:
from pyspark.sql.functions import when

orders_with_type = orders_df.withColumn("OrderType",when(col("Status") == "Returned", "Return").otherwise("Purchase"))
orders_with_type.show()

+-------+----------+---------+--------+-----+----------+---------+---------+
|OrderID|CustomerID|ProductID|Quantity|Price| OrderDate|   Status|OrderType|
+-------+----------+---------+--------+-----+----------+---------+---------+
|   3001|      C001|    P1001|       1|75000|2024-05-01|Delivered| Purchase|
|   3002|      C002|    P1002|       2|50000|2024-05-02| Returned|   Return|
|   3003|      C003|    P1003|       1|30000|2024-05-03|Delivered| Purchase|
|   3004|      C001|    P1002|       1|50000|2024-05-04|Delivered| Purchase|
|   3005|      C004|    P1004|       3|10000|2024-05-05|  Pending| Purchase|
+-------+----------+---------+--------+-----+----------+---------+---------+

